In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np

from itertools import combinations
from collections import Counter

In [2]:
df = pd.read_csv('hakaton.csv',sep=';')

In [3]:
seasons = {'winter': [1,2,12], 
           'spring': [3,4,5],
           'summer': [6,7,8],
           'autumn': [9,10,11]}

In [4]:
def preprocessing_df(df):
    df['create_datetime'] = pd.to_datetime(df['create_datetime'])
    df['order_day'] = df['create_datetime'].dt.day
    df['order_hour'] = df['create_datetime'].dt.hour
    df['order_weekday'] = df['create_datetime'].dt.weekday
    df['order_month'] = df['create_datetime'].dt.month
    df['order_year'] = df['create_datetime'].dt.year
    df['day_of_week_name'] = df['create_datetime'].dt.day_name()
    df['order_week'] = df['create_datetime'].dt.isocalendar().week
    #встречаются 1 раз
    #df_appear_once = df[df['entity_id'].isin(df['entity_id'].value_counts()[df['entity_id'].value_counts() == 1].index)]
    #удаление всего что встречается 1 раз
    #df = df[df['entity_id'].isin(df['entity_id'].value_counts()[df['entity_id'].value_counts() > 1].index)]
    return df

In [5]:
def buy_together(df):
    df_grouped = df.groupby(['order_id','customer_id'])['entity_id'].agg(list).reset_index()
    df_together = df_grouped[df_grouped['entity_id'].apply(len) >= 0]
    return df_together

In [6]:
def seasson_entitys(df):
    """
    выявление популярных товаров в течение дня 
    """
    
    df = df.groupby(['order_month','entity_id']).size().reset_index(name='count')
    return df

In [7]:
def daily_product(df, max_year = None, max_month = None, flag = None):
    """
    группировка по дням недели,возвращение товара дня по неделям (без потора товаров)
    """
    day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    max_year = df['order_year'].unique().max()
    max_month = df[df['order_year'] == max_year]['order_month'].unique().max()
    if max_month == 1: 
        max_year -= 1
        max_month = 12
    df_group_day_name_entity = df[(df['order_year'] == max_year) & (df['order_month'] == max_month)].groupby(['day_of_week_name', 'entity_id']).size().reset_index(name='count')
    df_group_day_name_entity = df_group_day_name_entity.sort_values(by = 'count', ascending=False).drop_duplicates(subset='entity_id', keep='first').drop_duplicates(subset='day_of_week_name', keep='first')[:7]
    df_group_day_name_entity['day_of_week_name'] = pd.Categorical(df_group_day_name_entity['day_of_week_name'], categories=day_order, ordered=True)
    df_group_day_name_entity = df_group_day_name_entity.sort_values('day_of_week_name').reset_index(drop=True)
    return df_group_day_name_entity

In [8]:
def find_most_frequent_pairs(df,customer_id = None):
    """
    Функция находит наиболее часто покупаемые товары вместе.
    
    :param df: DataFrame с колонками 'order_id' и 'entity_id' (где entity_id - список товаров)
    :return: DataFrame с парами товаров и их частотой
    """
    # Функция для генерации всех пар товаров из списка

    if customer_id:
        find_df = df[df['customer_id'] == customer_id]
    else:
        find_df = df

    # Функция для генерации всех пар товаров из списка
    def generate_item_pairs(item_list):
        return list(combinations(sorted(item_list), 2))

    # Генерация всех пар товаров для всех заказов
    all_pairs = []
    for item_list in find_df['entity_id']:
        all_pairs.extend(generate_item_pairs(item_list))

    # Подсчет частоты каждой пары товаров
    pair_counts = Counter(all_pairs)

    # Преобразование в DataFrame
    pair_counts_df = pd.DataFrame(pair_counts.items(), columns=['pair', 'count'])
    
    # Удаляем пары с одинаковыми значениями
    pair_counts_df = pair_counts_df[pair_counts_df['pair'].apply(lambda x: x[0] != x[1])]

    # Сортировка по частоте (count)
    pair_counts_df_sorted = pair_counts_df.sort_values(by='count', ascending=False).reset_index(drop=True)

    # Оставляем только пары с count >= 2
    pair_counts_df_sorted = pair_counts_df_sorted[pair_counts_df_sorted['count'] >= 2]

    # Если пар слишком мало (меньше 3), вызываем функцию повторно без customer_id
    if len(pair_counts_df_sorted) < 3:
        return find_most_frequent_pairs(df) 
    return pair_counts_df_sorted

In [9]:
def find_frequent_pairs_with_entity(df, entity_id):
    """
    Функция находит наиболее часто встречающиеся пары товаров с указанным entity_id,
    и возвращает только вторые товары в этих парах.
    
    :param df: DataFrame с колонками 'pair' и 'count' (где 'pair' - пара товаров, 'count' - частота)
    :param entity_id: Идентификатор товара, с которым ищем часто встречающиеся пары
    :return: DataFrame с вторыми товарами в парах и их частотой
    """
    filtered_df = df[df['pair'].apply(lambda x: entity_id in x)].copy()

    # Извлекаем вторые товары из каждой пары
    filtered_df['second_item'] = filtered_df['pair'].apply(lambda x: x[1] if x[0] == entity_id else x[0])
    
    # Сортируем по частоте (count) в порядке убывания
    filtered_df_sorted = filtered_df[['second_item', 'count']].sort_values(by='count', ascending=False).reset_index(drop=True)

    median_index = np.median(filtered_df_sorted.index)

    # Извлекаем индексы, которые находятся на расстоянии ±1 от медианы
    indices_near_median = filtered_df_sorted.index[(filtered_df_sorted.index >= median_index - 1) & (filtered_df_sorted.index <= median_index + 1)]

    # Создаем новый DataFrame с этими индексами
    filtered_df_sorted = filtered_df_sorted.loc[indices_near_median]
    
    return filtered_df_sorted[:3].loc[:,'second_item'].values

In [10]:
df = preprocessing_df(df)
df_daily_product = daily_product(df)
df_daily_product

,day_of_week_name,entity_id,count
0,Monday,44,40
1,Tuesday,542,4142
2,Wednesday,551,840
3,Thursday,579,254
4,Friday,557,65
5,Saturday,965,61
6,Sunday,711,20


In [11]:
df_season_entitys = seasson_entitys(df)

In [12]:
df_buy_together = buy_together(df)
df_buy_together

,order_id,customer_id,entity_id
0,1325312,9172,"[13, 379, 465]"
1,1331271,9173,[363]
2,1333417,9172,"[238, 25, 13]"
3,1336990,9178,[468]
4,1337295,9172,"[379, 13, 23]"
...,...,...,...
1628800,4391431,159858,"[571, 575]"
1628801,4391451,179242,"[592, 592]"
1628802,4391477,134172,"[551, 542]"
1628803,4391482,81065,"[605, 605, 475]"


In [13]:
most_frequent_pairs = find_most_frequent_pairs(df_buy_together, 9172)
most_frequent_pairs

,pair,count
0,"(238, 585)",10
1,"(584, 585)",9
2,"(350, 468)",7
3,"(238, 584)",7
4,"(536, 835)",5
...,...,...
115,"(13, 238)",2
116,"(462, 585)",2
117,"(467, 585)",2
118,"(238, 406)",2


In [14]:
take_entitys = find_frequent_pairs_with_entity(most_frequent_pairs, 585)
take_entitys

array([461, 536, 469], dtype=int64)

- Количество уникальных заказов на клиента (order_count).
- Количество уникальных позиций (товаров) на клиента (unique_entity_count).
- Среднее количество товаров в заказе (avg_items_per_order).
- Количество дней между заказами (interpurchase_time).
- Дни недели и время суток заказа (для выявления паттернов покупок).
- Вычисление размера корзины (если можно косвенно оценить количество товаров по entity_id).